In [ ]:
!pip install -r requirements.txt 

In [1]:
from transformers import AutoModel
import torch


torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = AutoModel.from_pretrained("/dataQ/zhijian/LLMs/chatglm-6b", trust_remote_code=True, device_map='auto')

/dataQ/zhijian/anaconda3/envs/chatglm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 8/8 [00:08<00:00,  1.10s/it]


In [2]:
from peft import PeftModel

model = PeftModel.from_pretrained(model, "../output/glm")
torch.set_default_tensor_type(torch.cuda.FloatTensor)

/dataQ/zhijian/anaconda3/envs/chatglm/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /dataQ/zhijian/anaconda3/envs/chatglm did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/dataQ/zhijian/anaconda3/envs/chatglm/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/dataQ/zhijian/anaconda3/envs/chatglm/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('"/dataQ/zhijian/.vscode-server-insiders/data/clp/0f47626ed20b541b4f5b69d0ee0ddde7.zh-cn","_resolvedLanguagePackCoreLocation"'), PosixPath('"zh-cn","osLocale"'), PosixPath('{"locale"'), PosixPath('{"*"'), PosixPath('"zh-cn","availableLanguages"'), PosixPath('"0f47626ed20b541b4


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 116
CUDA SETUP: Loading binary /dataQ/zhijian/anaconda3/envs/chatglm/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda116.so...


In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/dataQ/zhijian/LLMs/chatglm-6b", trust_remote_code=True)

In [ ]:
# 选取测试的 instructions，这里选取的是前三行
instructions = [
              {
                            "instruction": "战斗机涡轮前温度有什么用？",
                            "input": "",
                            "output": "原则上前涡轮温度升高后，燃烧更加充分，推力有所上升。但单纯增加涡轮前温会导致风扇寿命下降，WS10就出现过这种问题。"
              },
              {
                            "instruction": "上游二号是超音速反舰导弹吗",
                            "input": "",
                            "output": "不是，还是高亚音速的。上游2还是冥河的老底子，看看那个气动外形——卵型。就知道了，没法飞超音速。"
              },
              {
                            "instruction": "哪个国家激光武器最先进",
                            "input": "",
                            "output": "激光武器是一种利用定向发射的激光束直接毁伤目标或使之失效的定向能武器。武器系统主要由激光器和跟踪、瞄准、发射装置等部分组成。目前通常采用的激光器有化学激光器、固体激光器、二氧化碳激光器等。激光武器具有攻击速度快、转向灵活、可实现精确打击、不受电磁干扰等优点，但也存在昴受天气和环境影响等弱点。目前，激光器的种类繁多，名称各异，有体积整整占据一幢大楼、功率为上万亿瓦、用于引发核聚变的激光器，也有比人的指甲还小、输出功率仅有几毫瓦、用于光电通信的半导体激光器。按工作介质区分，可分为固体激光器、液体激光器和分子型、离子型、准分子型的气体激光器；按发射位置可分为天基、陆基、舰载、车载和机载等类型；按用途可分为战术型和战略型两类。打字不易，如满意。"
              }
]

In [7]:

def format_example(example: dict) -> dict:
    context = f"Instruction: {example['instruction']}\n"
    if example.get("input"):
        context += f"Input: {example['input']}\n"
    context += "Answer: "
    target = example["output"]
    return {"context": context, "target": target}


with torch.no_grad():
    for idx, item in enumerate(instructions):
        feature = format_example(item)
        input_text = feature['context']
        ids = tokenizer.encode(input_text)
        input_ids = torch.LongTensor([ids])
        out = model.generate(
            input_ids=input_ids,
            max_length=150,
            do_sample=False,
            temperature=0
        )
        out_text = tokenizer.decode(out[0])
        answer = out_text.replace(input_text, "").replace("\nEND", "").strip()
        item['infer_answer'] = answer
        print(out_text)
        print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')

Instruction: 战斗机涡轮前温度有什么用?
Answer: 1、涡轮前温度越高,涡轮叶片的温度也就越高,叶片能够承受的温度也就越高。2、涡轮前温度高,压气机的压强也就越高,飞机在压气机的基础上,能够利用的压强也就越高。3、飞机能够利用的压强越高,飞机的升力也就越高。
### 1.Answer:
 原则上前涡轮温度升高后，燃烧更加充分，推力有所上升。但单纯增加涡轮前温会导致风扇寿命下降，WS10就出现过这种问题。 


Instruction: 上游二号是超音速反舰导弹吗
Answer: 不是,是超音速反坦克导弹。上游2号(FS-2)反坦克导弹是前苏联于1980年研制成功的一种超音速反坦克导弹,于1981年装备部队,主要用来攻击装甲目标。上游2号导弹的射程、攻击精度和命中稳定性都比上游1号导弹有了很大提高。
### 2.Answer:
 不是，还是高亚音速的。上游2还是冥河的老底子，看看那个气动外形——卵型。就知道了，没法飞超音速。 


Instruction: 哪个国家激光武器最先进
Answer: 美国的,激光武器是冷战时期的产物,现在世界上很多国家都有激光武器,激光武器是一代新型武器,具有速度快、精度高、反应时间短、射程远等特点。激光武器具有速度快、精度高、反应时间短、射程远等特点,是新型高能激光武器的主要特点。目前,美国、俄罗斯、英国、法国、中国等5大国已拥有或正在研制10多种激光武器。美国“激光器专家”网站认为,激光武器具有潜在的优越性,认为激光武器是21世纪的“明星”激光武器具有潜在的优越性,即它既是一种有效的防御武器,又是一种有效的进攻武器。目前,激光武器已成为
### 3.Answer:
 激光武器是一种利用定向发射的激光束直接毁伤目标或使之失效的定向能武器。武器系统主要由激光器和跟踪、瞄准、发射装置等部分组成。目前通常采用的激光器有化学激光器、固体激光器、二氧化碳激光器等。激光武器具有攻击速度快、转向灵活、可实现精确打击、不受电磁干扰等优点，但也存在昴受天气和环境影响等弱点。目前，激光器的种类繁多，名称各异，有体积整整占据一幢大楼、功率为上万亿瓦、用于引发核聚变的激光器，也有比人的指甲还小、输出功率仅有几毫瓦、用于光电通信的半导体激光器。按工作介质区分，可分为固体激光器、液体激光器和分子型、离子型、准分子型的气体激光器；按发射位置可分为天